In [60]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from ipywidgets import interact, fixed
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils.multiclass import unique_labels
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import json
import glob
import os
import category_encoders as ce # this one needs "import sys" idk why
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from xgboost import plot_importance
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler



In [61]:
pd.set_option("display.max_rows", 101)

In [62]:
from google.colab import files
uploaded = files.upload()

Saving drugs and personality send to Tally.csv to drugs and personality send to Tally (1).csv


In [63]:
# let's have a general look at what the dataframe looks like
# whenever we are running df.replace we need to rerun this cell
df = pd.read_csv('drugs and personality send to Tally.csv')
print(df.shape)
df.head()

(1885, 32)


,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation,alcohol,amphetamines,amyl_nitrite_consumption,benzos,caffeine,cannabis,chocolate,cocaine,crack,ecstasy,heroin,katamine,legal_highs,lsd,meth,mushrooms,nicotine,semeron,volatile_substance_abuse
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [64]:
# when creating a dictionary for the categorical variables we want to see what we're working with
df.dtypes

id                            int64
age                         float64
gender                      float64
education                   float64
country                     float64
ethnicity                   float64
neuroticism                 float64
extraversion                float64
openness                    float64
agreeableness               float64
conscientiousness           float64
impulsiveness               float64
sensation                   float64
alcohol                      object
amphetamines                 object
amyl_nitrite_consumption     object
benzos                       object
caffeine                     object
cannabis                     object
chocolate                    object
cocaine                      object
crack                        object
ecstasy                      object
heroin                       object
katamine                     object
legal_highs                  object
lsd                          object
meth                        

In [65]:
# we want to limit the decimal places so that we can consistently replace in our 
# dictionary; as we will soon find out, some wonky things are happening behind 
# the scenes where we ultimately have to query specific observations from 
# specific features
df.round(decimals=5)

,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation,alcohol,amphetamines,amyl_nitrite_consumption,benzos,caffeine,cannabis,chocolate,cocaine,crack,ecstasy,heroin,katamine,legal_highs,lsd,meth,mushrooms,nicotine,semeron,volatile_substance_abuse
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,0.88113,1.92173,CL5,CL0,CL0,CL0,CL4,CL5,CL4,CL0,CL0,CL0,CL0,CL0,CL3,CL3,CL0,CL0,CL0,CL0,CL5
1881,1885,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.88113,0.76540,CL5,CL0,CL0,CL0,CL5,CL3,CL4,CL0,CL0,CL2,CL0,CL0,CL3,CL5,CL4,CL4,CL5,CL0,CL0
1882,1886,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,0.52975,-0.52593,CL4,CL6,CL5,CL5,CL6,CL6,CL6,CL4,CL0,CL4,CL0,CL2,CL0,CL2,CL0,CL2,CL6,CL0,CL0
1883,1887,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.29221,1.22470,CL5,CL0,CL0,CL0,CL6,CL6,CL5,CL0,CL0,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL4,CL0,CL0


In [66]:
# let's look at what specific values we need to replace
df['country'].value_counts()

 0.96082    1044
-0.57009     557
-0.28519     118
 0.24923      87
-0.09765      54
 0.21128      20
-0.46841       5
Name: country, dtype: int64

In [67]:
# get a general idea of numerical values of data
df.describe()

,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation
count,1885.000000,1885.00000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000
mean,945.294960,0.03461,-0.000256,-0.003806,0.355542,-0.309577,0.000047,-0.000163,-0.000534,-0.000245,-0.000386,0.007216,-0.003292
std,545.167641,0.87836,0.482588,0.950078,0.700335,0.166226,0.998106,0.997448,0.996229,0.997440,0.997523,0.954435,0.963701
min,1.000000,-0.95197,-0.482460,-2.435910,-0.570090,-1.107020,-3.464360,-3.273930,-3.273930,-3.464360,-3.464360,-2.555240,-2.078480
25%,474.000000,-0.95197,-0.482460,-0.611130,-0.570090,-0.316850,-0.678250,-0.695090,-0.717270,-0.606330,-0.652530,-0.711260,-0.525930
50%,946.000000,-0.07854,-0.482460,-0.059210,0.960820,-0.316850,0.042570,0.003320,-0.019280,-0.017290,-0.006650,-0.217120,0.079870
75%,1417.000000,0.49788,0.482460,0.454680,0.960820,-0.316850,0.629670,0.637790,0.723300,0.760960,0.584890,0.529750,0.765400
max,1888.000000,2.59171,0.482460,1.984370,0.960820,1.907250,3.273930,3.273930,2.901610,3.464360,3.464360,2.901610,1.921730


In [68]:
# we have 19 'categorical' variables
# these simply measure the user intensity for each 'drug'
print(df.describe(exclude='number').shape)
df.describe(exclude='number')

(4, 19)


,alcohol,amphetamines,amyl_nitrite_consumption,benzos,caffeine,cannabis,chocolate,cocaine,crack,ecstasy,heroin,katamine,legal_highs,lsd,meth,mushrooms,nicotine,semeron,volatile_substance_abuse
count,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885,1885
unique,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,5,7
top,CL5,CL0,CL0,CL0,CL6,CL6,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
freq,759,976,1305,1000,1385,463,807,1038,1627,1021,1605,1490,1094,1069,1429,982,610,1877,1455


In [ ]:
# list of categorical names; these are all of the 'drugs' being tested
df.describe(exclude='number').columns

Index(['alcohol', 'amphetamines', 'amyl_nitrite_consumption', 'benzos',
       'caffeine', 'cannabis', 'chocolate', 'cocaine', 'crack', 'ecstasy',
       'heroin', 'katamine', 'legal_highs', 'lsd', 'meth', 'mushrooms',
       'nicotine', 'semeron', 'volatile_substance_abuse'],
      dtype='object')

In [ ]:
print(df.describe(include='number').shape)
df.describe(include='number')

(8, 13)


,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation
count,1885.000000,1885.00000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000
mean,945.294960,0.03461,-0.000256,-0.003806,0.355542,-0.309577,0.000047,-0.000163,-0.000534,-0.000245,-0.000386,0.007216,-0.003292
std,545.167641,0.87836,0.482588,0.950078,0.700335,0.166226,0.998106,0.997448,0.996229,0.997440,0.997523,0.954435,0.963701
min,1.000000,-0.95197,-0.482460,-2.435910,-0.570090,-1.107020,-3.464360,-3.273930,-3.273930,-3.464360,-3.464360,-2.555240,-2.078480
25%,474.000000,-0.95197,-0.482460,-0.611130,-0.570090,-0.316850,-0.678250,-0.695090,-0.717270,-0.606330,-0.652530,-0.711260,-0.525930
50%,946.000000,-0.07854,-0.482460,-0.059210,0.960820,-0.316850,0.042570,0.003320,-0.019280,-0.017290,-0.006650,-0.217120,0.079870
75%,1417.000000,0.49788,0.482460,0.454680,0.960820,-0.316850,0.629670,0.637790,0.723300,0.760960,0.584890,0.529750,0.765400
max,1888.000000,2.59171,0.482460,1.984370,0.960820,1.907250,3.273930,3.273930,2.901610,3.464360,3.464360,2.901610,1.921730


In [ ]:
# we have 13 numerical variables
df.describe(include='number').columns

Index(['id', 'age', 'gender', 'education', 'country', 'ethnicity',
       'neuroticism', 'extraversion', 'openness', 'agreeableness',
       'conscientiousness', 'impulsiveness', 'sensation'],
      dtype='object')

In [ ]:
df['gender'].describe()

count    1885.000000
mean       -0.000256
std         0.482588
min        -0.482460
25%        -0.482460
50%        -0.482460
75%         0.482460
max         0.482460
Name: gender, dtype: float64

In [69]:
categorical_df = df.select_dtypes(exclude='number').copy()
categorical_df

,alcohol,amphetamines,amyl_nitrite_consumption,benzos,caffeine,cannabis,chocolate,cocaine,crack,ecstasy,heroin,katamine,legal_highs,lsd,meth,mushrooms,nicotine,semeron,volatile_substance_abuse
0,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,CL5,CL0,CL0,CL0,CL4,CL5,CL4,CL0,CL0,CL0,CL0,CL0,CL3,CL3,CL0,CL0,CL0,CL0,CL5
1881,CL5,CL0,CL0,CL0,CL5,CL3,CL4,CL0,CL0,CL2,CL0,CL0,CL3,CL5,CL4,CL4,CL5,CL0,CL0
1882,CL4,CL6,CL5,CL5,CL6,CL6,CL6,CL4,CL0,CL4,CL0,CL2,CL0,CL2,CL0,CL2,CL6,CL0,CL0
1883,CL5,CL0,CL0,CL0,CL6,CL6,CL5,CL0,CL0,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL4,CL0,CL0


In [70]:
df['country'].value_counts()

 0.96082    1044
-0.57009     557
-0.28519     118
 0.24923      87
-0.09765      54
 0.21128      20
-0.46841       5
Name: country, dtype: int64

In [71]:
cleanup_levels = {'age':{2.59171:5,1.82213:4,1.09449:3,0.49788000000000004:2,
                         -0.07854:1, -0.9519700000000001:0},
                  'gender':{0.48246000000000006:'Female', -0.48246000000000006:'Male'},
                  'education':{1.98437:8,1.16365:7,0.45468000000000003:6,-0.-0.059210000000000006:5,
                               -0.6111300000000001:4,-1.22751:3,-1.43719:2,
                               -1.73790:1, -2.43591:0},
                  'country':{-0.09765: 'Australia',0.24923:"Canada",
                             -0.46841000000000005:'New Zealand',-0.28519:"Other",
                             0.21128000000000002:'Republic of Ireland',0.9608200000000001:'UK',
                             -0.57009:'USA'},
                  'ethnicity':{-0.31685:6,0.11440:5,-1.10702:4,
                               -0.50212:3,-0.22166:2,0.12600:1, 
                               1.90725:0},
                  'alcohol':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                             'CL0':0},
                  'amphetamines':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,
                                  'CL1':1, 'CL0':0},
                  'amyl_nitrite_consumption':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,
                                              'CL2':2,'CL1':1, 'CL0':0},
                  'benzos':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                            'CL0':0},
                  'caffeine':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                              'CL0':0},
                  'cannabis':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                              'CL0':0},
                  'cocaine':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                             'CL0':0},
                  'crack':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                           'CL0':0},
                  'ecstasy':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                             'CL0':0},
                  'katamine':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 
                              'CL0':0},
                  'legal_highs':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'lsd':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'meth':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'mushrooms':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'nicotine':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'semeron':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'volatile_substance_abuse':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'heroin':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0},
                  'chocolate':{'CL6':6,'CL5':5,'CL4':4,'CL3':3,'CL2':2,'CL1':1, 'CL0':0}}

In [72]:
# this applies the dictionary to my dataframe
df.replace(cleanup_levels, inplace=True) # only run once

In [73]:
# check that it has been applied properly
df.head(20)

,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation,alcohol,amphetamines,amyl_nitrite_consumption,benzos,caffeine,cannabis,chocolate,cocaine,crack,ecstasy,heroin,katamine,legal_highs,lsd,meth,mushrooms,nicotine,semeron,volatile_substance_abuse
0,1,2.0,Female,5.0,UK,1.0,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,5,2,0,2,6,0,5,0,0,0,0,0,0,0,0,0,2,0,0
1,2,1.0,Male,8.0,UK,6.0,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,5,2,2,0,6,4,6,3,0,4,0,2,0,2,3,0,4,0,0
2,3,2.0,Male,5.0,UK,6.0,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,6,0,0,0,6,3,4,0,0,0,0,0,0,0,0,1,0,0,0
3,4,0.0,Female,7.0,UK,6.0,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,4,0,0,3,5,2,4,2,0,0,0,2,0,0,0,0,2,0,0
4,5,2.0,Female,8.0,UK,6.0,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,4,1,1,0,6,3,6,0,0,1,0,0,1,0,0,2,2,0,0
5,6,5.0,Female,3.0,Canada,6.0,-0.67825,-0.30033,-1.55521,2.03972,1.63088,-1.37983,-1.54858,2,0,0,0,6,0,4,0,0,0,0,0,0,0,0,0,6,0,0
6,7,3.0,Male,7.0,USA,6.0,-0.46725,-1.09207,-0.45174,-0.30172,0.93949,-0.21712,0.07987,6,0,0,0,6,1,5,0,0,0,0,0,0,0,0,0,6,0,0
7,8,2.0,Male,1.0,UK,6.0,-1.32828,1.93886,-0.84732,-0.30172,1.63088,0.19268,-0.52593,5,0,0,0,6,0,4,0,0,0,0,0,0,0,0,0,0,0,0
8,9,2.0,Female,5.0,Canada,6.0,0.62967,2.57309,-0.97631,0.76096,1.13407,-1.37983,-1.54858,4,0,0,0,6,0,6,0,0,0,0,0,0,0,0,0,6,0,0
9,10,4.0,Male,7.0,UK,6.0,-0.24649,0.00332,-1.42424,0.59042,0.12331,-1.37983,-0.84637,6,1,0,1,6,1,6,0,0,0,0,0,0,0,0,0,6,0,0


In [74]:
# you have to pull the exact value; 
# we have modified again and again until it shows the correct values
df.age[3]

0.0

In [75]:
df.country.value_counts()

UK                     1044
USA                     557
Other                   118
Canada                   87
Australia                54
Republic of Ireland      20
New Zealand               5
Name: country, dtype: int64

In [76]:
# we see which floats have been successfully changed; we have successfully
# changed some numeric values to categorical ones and categoricals (CL0 to CL6)
# to simple integers
df.dtypes

id                            int64
age                         float64
gender                       object
education                   float64
country                      object
ethnicity                   float64
neuroticism                 float64
extraversion                float64
openness                    float64
agreeableness               float64
conscientiousness           float64
impulsiveness               float64
sensation                   float64
alcohol                       int64
amphetamines                  int64
amyl_nitrite_consumption      int64
benzos                        int64
caffeine                      int64
cannabis                      int64
chocolate                     int64
cocaine                       int64
crack                         int64
ecstasy                       int64
heroin                        int64
katamine                      int64
legal_highs                   int64
lsd                           int64
meth                        

In [77]:
core = df.drop(['amphetamines','amyl_nitrite_consumption','benzos','crack',
                'ecstasy','heroin','katamine','meth','semeron',
                'volatile_substance_abuse'], axis=1)

In [78]:
# our initial dataframe has 33 columns but we have reduced it to 22
print(core.info())
core.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885 entries, 0 to 1884
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1885 non-null   int64  
 1   age                1885 non-null   float64
 2   gender             1885 non-null   object 
 3   education          1885 non-null   float64
 4   country            1885 non-null   object 
 5   ethnicity          1885 non-null   float64
 6   neuroticism        1885 non-null   float64
 7   extraversion       1885 non-null   float64
 8   openness           1885 non-null   float64
 9   agreeableness      1885 non-null   float64
 10  conscientiousness  1885 non-null   float64
 11  impulsiveness      1885 non-null   float64
 12  sensation          1885 non-null   float64
 13  alcohol            1885 non-null   int64  
 14  caffeine           1885 non-null   int64  
 15  cannabis           1885 non-null   int64  
 16  chocolate          1885 

,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation,alcohol,caffeine,cannabis,chocolate,cocaine,legal_highs,lsd,mushrooms,nicotine
0,1,2.0,Female,5.0,UK,1.0,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,5,6,0,5,0,0,0,0,2
1,2,1.0,Male,8.0,UK,6.0,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,5,6,4,6,3,0,2,0,4
2,3,2.0,Male,5.0,UK,6.0,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,6,6,3,4,0,0,0,1,0
3,4,0.0,Female,7.0,UK,6.0,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,4,5,2,4,2,0,0,0,2
4,5,2.0,Female,8.0,UK,6.0,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,4,6,3,6,0,1,0,2,2


In [81]:
corecountries1 = core[~core.country.str.contains('New Zealand')]
corecountries1.country.value_counts()

UK                     1044
USA                     557
Other                   118
Canada                   87
Australia                54
Republic of Ireland      20
Name: country, dtype: int64

In [82]:
corecountries = core[~core.country.str.contains('Republic of Ireland')]
corecountries.country.value_counts()

UK           1044
USA           557
Other         118
Canada         87
Australia      54
Name: country, dtype: int64

In [83]:
corecountries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1860 entries, 0 to 1883
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1860 non-null   int64  
 1   age                1860 non-null   float64
 2   gender             1860 non-null   object 
 3   education          1860 non-null   float64
 4   country            1860 non-null   object 
 5   ethnicity          1860 non-null   float64
 6   neuroticism        1860 non-null   float64
 7   extraversion       1860 non-null   float64
 8   openness           1860 non-null   float64
 9   agreeableness      1860 non-null   float64
 10  conscientiousness  1860 non-null   float64
 11  impulsiveness      1860 non-null   float64
 12  sensation          1860 non-null   float64
 13  alcohol            1860 non-null   int64  
 14  caffeine           1860 non-null   int64  
 15  cannabis           1860 non-null   int64  
 16  chocolate          1860 

In [84]:
1860*.8

1488.0

In [86]:
corecountries.id[1488]

1492

In [87]:
train_data = corecountries[df.id < 1492]
test_data  = corecountries[df.id >= 1492]

# cutoff = pd.to_datetime('2018-10-01')
# train_data = df[df.game_date <= cutoff]
# test_data  = df[df.game_date > cutoff]

test = test_data
train = train_data
target = 'cocaine'
train, val = train_test_split(train, test_size=len(test),  
                              stratify=train[target], random_state=42)
train_features = train.drop(columns=[target])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the features 
features = numeric_features + categorical_features
X_val = val[features]
y_val = val[target]
print(features)

['id', 'age', 'education', 'ethnicity', 'neuroticism', 'extraversion', 'openness', 'agreeableness', 'conscientiousness', 'impulsiveness', 'sensation', 'alcohol', 'caffeine', 'cannabis', 'chocolate', 'legal_highs', 'lsd', 'mushrooms', 'nicotine', 'gender', 'country']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
print(train_data.shape)
train_data.head()

(1508, 22)


,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation,alcohol,caffeine,cannabis,chocolate,cocaine,legal_highs,lsd,mushrooms,nicotine
0,1,2.0,Female,5.0,UK,1.0,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,5,6,0,5,0,0,0,0,2
1,2,1.0,Male,8.0,UK,6.0,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,5,6,4,6,3,0,2,0,4
2,3,2.0,Male,5.0,UK,6.0,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,6,6,3,4,0,0,0,1,0
3,4,0.0,Female,7.0,UK,6.0,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,4,5,2,4,2,0,0,0,2
4,5,2.0,Female,8.0,UK,6.0,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,4,6,3,6,0,1,0,2,2


In [ ]:
train_data['country'].value_counts()

UK                     849
USA                    418
Other                  100
Canada                  76
Australia               45
Republic of Ireland     16
New Zealand              4
Name: country, dtype: int64

In [ ]:
print(test_data.shape)
test_data.head()

(377, 22)


,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness,agreeableness,conscientiousness,impulsiveness,sensation,alcohol,caffeine,cannabis,chocolate,cocaine,legal_highs,lsd,mushrooms,nicotine
1508,1512,0.0,Male,6.0,USA,6.0,0.73545,-0.80615,1.65653,0.13136,-1.25773,-0.21712,-0.52593,5,6,6,6,5,4,3,2,6
1509,1513,0.0,Male,0.0,USA,6.0,2.28554,-0.69509,0.29338,-0.60633,-0.14277,-1.37983,-0.84637,6,4,6,5,0,2,0,4,3
1510,1514,0.0,Male,4.0,USA,1.0,-0.14882,-0.57545,-0.84732,-0.60633,-0.52745,-0.21712,-2.07848,3,6,6,3,2,2,2,2,6
1511,1515,1.0,Male,3.0,USA,6.0,1.23461,-2.32338,0.29338,-0.30172,-1.92173,0.19268,-0.52593,3,6,2,6,2,2,2,2,6
1512,1516,0.0,Male,4.0,USA,6.0,1.02119,-0.43999,-0.01928,-1.34289,-1.13788,1.86203,1.22470,6,6,4,6,4,3,0,0,5


In [ ]:
test_data['country'].value_counts()

UK                     195
USA                    139
Other                   18
Canada                  11
Australia                9
Republic of Ireland      4
New Zealand              1
Name: country, dtype: int64